# Quasiperiodic Lorentz Gas

In this notebook we will see how the apply the code contained in the file "Magnetorresistencia_En_Cuasicristales.jl" to study the diffusion of an electron in a 2D plate doped with antiquantum dots in the presence of an external magnetic field. To do that we use a generalization of the "Cut-and-Project" method.

First, let's load the file that contain all the code that we will use in this notebook.

In [1]:
include("Magnetorresistencia_En_Cuasicristales.jl");

dibuja_Circulo (generic function with 2 methods)

To visualize our results we will use the "Plots" package, so let's load it.

In [2]:
using Plots

### Configuration of the system

Let's generate the parameters that will define our system:

In [51]:
Box_Size_Length = 1.0; #The length of one side of the cubic box centered at the origin that will contain the cylinder.
Cylinder_Axis = [π*0.008, sqrt(2)*0.008, 1]; #Vector that point into the direction of the cylinder axis.
Cylinder_Axis = Cylinder_Axis/norm(Cylinder_Axis); #Let's normalize the vector Cylinder_Axis.
Trajectory_Radius = 0.506; #Radius of the circular trajectory that follows our particle.

Z_to_Cyl_Matrix = mat_Rot(Cylinder_Axis); #Generate the matrix that rotates the vector (0,0,1) to the vector "Cylinder_Axis"
Cyl_to_Z_Matrix = (Z_to_Cyl_Matrix)^-1; #Generate the matrix that rotates the vector "Cylinder_Axis" to the vector (0,0,1)
Cylinder_Radius = 0.28; #Define the radius of the cylinder inside the unitary cube.

#Now let's calculates the length of the cylinder. This length will be determinated as the maximum distance between the
#projections to the "infinite" cylinder of the vertices of the cube.
Cylinder_Length = longitud_Cilindro(Box_Size_Length, Cyl_to_Z_Matrix);

### Algorithm for the Mean Square Displacement of a particle

In [ ]:
Iterations_Number = 10; #Number of trajectories that we will consider.

MSD_Array = []; #An array that will contain the Mean Square Displacement of the different trajectories.
TX = Array[]; #An array that will contain the X coordinates of one trajectory
TY = Array[]; #An array that will contain the Y coordinates of one trajectory
Centers_D = Array[]; #I HAVE NO IDEA FOR WHAT IT'S THIS ARRAY.
Initial_Position_X_Array = Array[]; #An array that will contain the X coordinates of the different initial positions
Initial_Position_Y_Array = Array[]; #An array that will contain the Y coordinates of the different initial positions

for i in 1:Iterations_Number
    @show i #We show to the user the actual step of the iterations.
    θ = π*rand() - π/2; #Let's define one angle between -π/2 and π/2.
    θ2 = π*rand(); #Let's define one angle between 0 and π.
    Height = rand()-0.5; #Height of the initial position of the particle.
    N_Total = zeros(Float64, 3); #I HAVE NO IDEA FOR WHAT IT'S THIS PARAMETER.
    Collision_Position = [Cylinder_Radius*cos(θ2), Cylinder_Radius*sin(θ2), 0.0]; #Position projected in the plane XY after the cylinder is vertical associated with the point where the particle collide with the cylinder.
    Normalized_Collision_Position = Collision_Position/norm(Collision_Position); #Let's normalize the vector Collision_Position.
    Velocity = [cos(θ),sin(θ),0.0]; #Velocity vector in the plane XY with arbitrary direction between the quadrants [IV,I]
    Orthogonal_Velocity = (Normalized_Collision_Position⋅Velocity)*[Normalized_Collision_Position[1], Normalized_Collision_Position[2], 0.0]; #Orthogonal component of the velocity of the particle after the collision.
    Velocity_After_Collision = Velocity - 2*Orthogonal_Velocity; #Tangent component of the velocity of the particle after the collision.
    Velocity_After_Collision = Z_to_Cyl_Matrix*Velocity_After_Collision; #Get this component of the velocity after the collision in the coordinates where the cylinder is not vertical.
    
    Initial_Velocity = Velocity_After_Collision; #Let's set the initial velocity of our particle as the velocity after one collision in the coordinates where the cylinder is not vertical.
    Initial_Position = Z_to_Cyl_Matrix*(Collision_Position + [0.0, 0.0, Height]) - Initial_Velocity * 0.000000001; #Let's set the initial position of our particle as the position an instant after the collision.
    
    Initial_Position_Cyl_Vertical = Cyl_to_Z_Matrix*Initial_Position; #Initial position of our particle in the coordinates where the cylinder is vertical.
    push!(Initial_Position_X_Array, [Initial_Position_Cyl_Vertical[1]]); #Let's push the X coordinate of the initial position in the space where the cylinder is vertical into the corresponding array.
    push!(Initial_Position_Y_Array, [Initial_Position_Cyl_Vertical[2]]); #Let's push the Y coordinate of the initial position in the space where the cylinder is vertical into the corresponding array.
    
    Fly_Time = 10; #Determine the length of the whole trajectory of the particle.
    Initial_Velocity /= norm(Initial_Velocity); #Let's normalize the initial velocity of our particle.
    
    Centers = []; #I HAVE NO IDEA FOR WHAT IT'S THIS ARRAY.
    Tx = []; #I HAVE NO IDEA FOR WHAT IT'S THIS ARRAY.
    Ty= []; #I HAVE NO IDEA FOR WHAT IT'S THIS ARRAY.
    CEN = Array[]; #I HAVE NO IDEA FOR WHAT IT'S THIS ARRAY.